在开始介绍 pytorch 的一个使用语法结构前,我们先说一下 深度学习领域几位真正的大神,可以说深度学习领域的目前的产出都离不开这几位大神
Hinton, LeCun, Bengio

Hinton 杰弗里·埃弗里斯特·辛顿
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/34/Geoffrey_Hinton_at_UBC.jpg/440px-Geoffrey_Hinton_at_UBC.jpg" style="zoom:50%;margin-left:0px" />

杰弗里·埃弗里斯特·辛顿 FRS（英语：Geoffrey Everest Hinton）（1947年12月6日－）是一位英国出生的加拿大计算机学家和心理学家，以其在神经网络方面的贡献闻名。辛顿是反向传播算法和对比散度算法的发明人之一，也是深度学习的积极推动者

后面两位就不细说了,总之都是很牛,可以说这三位以及他们的学生 点亮了深度学习领域的一盏明灯







### pytorch 基础及重要 api 介绍

首先 推荐优先使用虚拟环境,接下来我们准备安装依赖包 

```
pip3 install http://download.pytorch.org/whl/torch-0.3.1-cp36-cp36m-macosx_10_7_x86_64.whl 
pip3 install torchvision 
```

#### Tensor 张量
Pytorch 里面的基本对象就是张量 Tensor, 其表示的是已给多维的矩阵,其和 numpy 的数组是对应的 Tensor ,二者的区别就是 Pytorch 的 Tensor 可以运行在 GPU 上, TensorFlow 的 tensor 也是

依据数据类型 tensor 分一下几种
- torch.FloatTensor 32
- torch.DoubleTensor 64
- torch.shortTensor 16
- torch.IntTensor 32

下面我们给一些实例说明演示一下

In [1]:
import torch
a = torch.FloatTensor([[2,3], [4,8], [7,9]])
print("{}".format(a))
print("a size is {}".format(a.size()))


 2  3
 4  8
 7  9
[torch.FloatTensor of size 3x2]

a size is torch.Size([3, 2])


In [9]:
#使用 torch 创建全是 0 的张量
c = torch.zeros((3,2))
print("zero tensor:{}".format(c))

#随机初始化指定矩阵
d = torch.randn((3,2))
print("randn init the tensor:{}".format(d))

zero tensor:
 0  0
 0  0
 0  0
[torch.FloatTensor of size 3x2]

randn init the tensor:
-0.2921  0.8625
 0.2409 -0.4369
 0.2295  1.0684
[torch.FloatTensor of size 3x2]



In [13]:
import numpy as np
#与 numpy 互转
numpy_b = d.numpy()
print("conver to numpy is \n {}".format(numpy_b))
#np array conver to tensor
e = np.array([[2, 3], [4, 5]])
torch_e = torch.from_numpy(e)
print("the torch_e is : {}".format(torch_e))
print("the torch_e change data type is : {}".format(torch_e.float()))

conver to numpy is 
 [[-0.29212478  0.86247635]
 [ 0.24088798 -0.4369324 ]
 [ 0.22953479  1.0684093 ]]
the torch_e is : 
 2  3
 4  5
[torch.LongTensor of size 2x2]

the torch_e change data type is : 
 2  3
 4  5
[torch.FloatTensor of size 2x2]



#### Variable 
变量的本质其实和张量相同,只不过 torch 当中的变量可以自动求导,不过变量不会被放入计算图中, torch 的变量实在 torch.autograd.Variable 中 将一个 tensor 变成 Variable 很简单 只需要 Variable(a) 就可以了
> Variable 包含三个比较重要的组成属性: ***data*** , ***grad*** , ***grad_fn*** 通过 data 可以取出变量里面的张量数值, grad_fn 是计算梯度的操作,比如是通过加减还是乘除得来的,最后 grad 是反向传播的梯度

In [17]:
from torch.autograd import Variable

x = Variable(torch.Tensor([1]), requires_grad = True)
w = Variable(torch.Tensor([2]), requires_grad = True)
b = Variable(torch.Tensor([3]), requires_grad = True)
# Build a computational graph
y = w * x + b   # y = 2 * x +3

y.backward()

print(x.grad)    # x.grad = 2
print(w.grad)    # w.grad = 1
print(b.grad)    # b.grad = 1

Variable containing:
 2
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]



In [19]:
# 除了上面针对标量求导以外,我们还可以针对矩阵求导
x = torch.randn(3)
x = Variable(x, requires_grad=True)
y = x * 2
print("y:{}".format(y))
y.backward(torch.FloatTensor([1, 0.1, 0.01]))
print(x.grad)

y:Variable containing:
 0.6424
 1.6938
-0.5470
[torch.FloatTensor of size 3]

Variable containing:
 2.0000
 0.2000
 0.0200
[torch.FloatTensor of size 3]



#### 数据集
在处理任何机器学习问题之前都需要数据读取,并进行预处理. Pytorch 提供了很多工具使得数据的读取和预处理变得很容易.

torch.utils.data.DataSet 是代表这一数据的抽象类,你可以自己定义你的数据类继承 和 重写 这个抽象类, 你可以自己定义你的数据类继承 和重写这个抽象类,非常简单, 只需要定义 ***_ _len_ _*** ,***_ _getitem_ _*** 这两个列表,例如

In [21]:
from torch.utils.data import Dataset,DataLoader
class MyDataSet(Dataset):
    def __init__(self, csv_file, txt_file, root_dir, other_file):
        self.csv_file = csv_file
        with open(txt_file,'r') as f:
            data_list = f.readlines()
        self.txt_data = data_list
    
    
    def __len__(self):
        return len(self.csv_file)
    
    
    def __getitem__(self, idx):
        data = txt_data[idx]
        return data

In [23]:
# 批量迭代可以使用 
#dataiter = DataLoader(MyDataSet, batch_size=32, shuffle=True, collate_fn=default_collate)

#### 模组 nn.Module
在 pyTorch 里面编写神经网络,所有的层结构 和损失函数都来自于 torch.nn, 所有的模型都从这个基类继承而来  nn.Module

In [ ]:
class net_name(torch.nn.Module):
    def __init__(self, other_arguments):
        super("net_name", self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=None, out_channels=None, kernel_size=3)
        
        
    def forward(self, x):
        x = self.conv1
        return x